In [1]:
#| default_exp plotting

In [2]:
#| export
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import json


In [3]:
#| export

def imshow(im, figsize=(9, 9), ticks=False, **kwargs):
    plt.figure(figsize=figsize)
    if ticks is False:
        plt.xticks([])
        plt.yticks([])
    plt.imshow(im)


def image_grid(images, shape, figsize=(18, 9)):
    assert len(images) == np.prod(shape)
    fig, axes = plt.subplots(*shape, figsize=figsize)
    axes = axes.reshape(shape)
    rows, columns = shape
    for x in range(rows):
        for y in range(columns):
            i = x * columns + y
            axes[x, y].imshow(images[i])
            axes[x, y].set_xticks([])
            axes[x, y].set_yticks([])


In [5]:
#| export

def draw_keypoints(image, keypoints, size=3, color=(255, 0, 0)):
    image = image.copy()
    for x, y in keypoints:
        x = int(x)
        y = int(y)
        image[x - size : x + size, y - size : y + size] = color
    return image

In [6]:
#| hide
import nbdev
nbdev.nbdev_export()